Using Counting Vesicular Release Events Reveals Binomial Release Statistics at Single Glutamatergic Synapses by Malagon, Miki, Llano, Neher & Marty (The Journal of Neuroscience, 2016) and orking with a Gaussian approach.

In [ ]:
import math
from math import comb
from scipy.stats import norm

# ----------------------------
# Parameters based on Malagon et al.
# ----------------------------
#  baseline release probability (first stimulus, 1.5 mM Ca) as null hypothesis
n = 14               # suggested number of trials
p_null = 0.089       # baseline release probability from Malagon et al. (first stimulus, physiological Ca) 
alpha = 0.05

# ----------------------------
# New experimental observation
# ----------------------------
k_obs = 7            # hypothesized 7 successes

# ----------------------------
# Binomial exact test
# ----------------------------
def binom_pmf(n, k, p):
    return comb(n, k) * (p**k) * ((1-p)**(n-k))

def binom_p_value_exact(n, k_obs, p):
    # one-sided P(X >= k_obs)
    p_ge = sum(binom_pmf(n, k, p) for k in range(k_obs, n+1))
    # two-sided by sum of probabilities <= observed
    probs = [binom_pmf(n, k, p) for k in range(n+1)]
    p_obs = binom_pmf(n, k_obs, p)
    p_two = sum(p for p in probs if p <= p_obs)
    return p_ge, p_two

# ----------------------------
# Gaussian approximation (normal approx to binomial)
# ----------------------------
mu = n * p_null
sigma = math.sqrt(n * p_null * (1 - p_null))

z = (k_obs - mu) / sigma
p_one_sided_norm = 1 - norm.cdf(z)
p_two_sided_norm = 2 * (1 - norm.cdf(abs(z)))

# ----------------------------
# Results
# ----------------------------
p_exact = binom_pmf(n, k_obs, p_null)
p_one_exact, p_two_exact = binom_p_value_exact(n, k_obs, p_null)

print("Using Malagon et al. baseline (p_null = {:.3f})".format(p_null))
print("--- New Measurement: k_obs = {}/{} ---".format(k_obs, n))
print(f"Exact binomial probability of exactly {k_obs} successes: {p_exact:.6e}")
print(f"One-sided p-value (P(X >= {k_obs})): {p_one_exact:.6f}")
print(f"Two-sided p-value: {p_two_exact:.6f}")

print("\nGaussian approximation:")
print(f"Expected mean under null = {mu:.3f}, SD = {sigma:.3f}")
print(f"z-score = {z:.3f}")
print(f"One-sided p (norm approx) ≈ {p_one_sided_norm:.6f}")
print(f"Two-sided p (norm approx) ≈ {p_two_sided_norm:.6f}")

if p_two_exact < alpha:
    print("\n-> Two-sided test: reject null at alpha = {:.2f}".format(alpha))
else:
    print("\n-> Two-sided test: fail to reject null at alpha = {:.2f}".format(alpha))